In [1]:
from IPython.core.interacticeshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

ModuleNotFoundError: No module named 'IPython.core.interacticeshell'

In [2]:
import datarobot as dr

In [3]:
dr.Client(config_path = 'drconfig.yaml')

In [5]:
import pandas as pd

In [7]:
trainingData = pd.read_csv('./winequality-white-training.csv')
trainingData.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [11]:
from datetime import date
projectName = 'Pyhton Wine Quality '+ date.today().strftime(format = '%m-%d-%Y')
print(projectName)

Pyhton Wine Quality 07-13-2023


In [14]:
project = dr.Project.create (
    sourcedata = trainingData,
    project_name = projectName
)

print(project.id,project.project_name)

64af92710719c2ee6f0d815c Pyhton Wine Quality 07-13-2023


In [15]:
project.get_features()

[Feature(alcohol),
 Feature(chlorides),
 Feature(citric acid),
 Feature(density),
 Feature(fixed acidity),
 Feature(free sulfur dioxide),
 Feature(pH),
 Feature(quality),
 Feature(residual sugar),
 Feature(sulphates),
 Feature(total sulfur dioxide),
 Feature(volatile acidity)]

In [18]:
project.set_target(target = 'quality')

C:\Users\huzai\AppData\Local\Temp\ipykernel_6652\3688222327.py:1: DataRobotDeprecationWarning: `set_target` has been deprecated in `v3.0`, will be removed in `v4.0`. This method, 'set_target' is deprecated. Please use 'analyze_and_model' instead.
  project.set_target(target = 'quality')


ClientError: 422 client error: {'message': 'Target quality was already selected.'}

In [19]:
project.wait_for_autopilot()

In progress: 4, queued: 10 (waited: 0s)
In progress: 4, queued: 10 (waited: 1s)
In progress: 3, queued: 10 (waited: 3s)
In progress: 3, queued: 10 (waited: 4s)
In progress: 3, queued: 10 (waited: 7s)
In progress: 3, queued: 10 (waited: 10s)
In progress: 3, queued: 10 (waited: 14s)
In progress: 3, queued: 9 (waited: 22s)
In progress: 4, queued: 8 (waited: 36s)
In progress: 3, queued: 7 (waited: 58s)
In progress: 3, queued: 6 (waited: 79s)
In progress: 4, queued: 4 (waited: 100s)
In progress: 3, queued: 2 (waited: 121s)
In progress: 4, queued: 0 (waited: 143s)
In progress: 2, queued: 0 (waited: 164s)
In progress: 1, queued: 0 (waited: 185s)
In progress: 0, queued: 0 (waited: 206s)
In progress: 0, queued: 0 (waited: 228s)
In progress: 0, queued: 0 (waited: 249s)


KeyboardInterrupt: 

# Viewing the Model

In [4]:
for p in dr.Project.list():
    print(p.id, p.project_name)

64b77fbf94f1351b5332d3c6 LLR655 2022.csv
64b765d1231a4cdacc32d310 LLR655 sept-dec 2022.csv
64b6a5b2d2d6d87ae6f7cc73 sept2022.csv
64b3c1b36873af5a68da1835 aug_train.csv
64af9b081ab5359d6eddae3e Pyhton Wine Quality 07-13-2023
64af92710719c2ee6f0d815c Pyhton Wine Quality 07-13-2023
64af92333d27e33c3b0d83dd Pyhton Wine Quality 07-13-2023
64a379be00ad460c05854ffc StackOverflow_Advanced.csv
64a36ebb3f50357469b7c9be StackOverflow.csv
64a35ea39a6b1dc86985506d DR_Demo_LendingClub_Guardrails.csv
64a2ec92d31b8eb7ee854fc9 diabetes.csv


In [7]:
projectId = '64af92710719c2ee6f0d815c'
project = dr.Project.get(projectId)
print(project)

Project(Pyhton Wine Quality 07-13-2023)


In [8]:
models = project.get_models()
for m in models:
    print(m.id, m.model_type)

64af95280836eb930a03fe07 RandomForest Regressor
64af9306fe57f0cf4f00dd3e RandomForest Regressor
64af9306fe57f0cf4f00dd3c Light Gradient Boosted Trees Regressor with Early Stopping
64af9306fe57f0cf4f00dd40 eXtreme Gradient Boosted Trees Regressor
64af9306fe57f0cf4f00dd3d Light Gradient Boosting on ElasticNet Predictions 
64af9306fe57f0cf4f00dd3f RuleFit Regressor
64af9306fe57f0cf4f00dd3b Generalized Additive2 Model
64af9306fe57f0cf4f00dd39 Ridge Regressor
64af9306fe57f0cf4f00dd3a Elastic-Net Regressor (mixing alpha=0.5 / Least-Squares Loss)


In [9]:
recommendedModel = dr.ModelRecommendation.get(project.id).get_model()
print(recommendedModel, recommendedModel.model_type)

Model('RandomForest Regressor') RandomForest Regressor


# Deploying the Model

In [11]:
deployment = dr.Deployment.create_from_learning_model(
    model_id = recommendedModel.id,
    label = 'Wine Quality',
    description = 'Model for scoring wine quality'
)

# Predictions

In [12]:
job = dr.BatchPredictionJob.score (
    deployment=deployment.id,
    passthrough_columns=['wine_id'],
    intake_settings={
        'type': 'localFile',
        'file': './winequality-white-score.csv'
    },
    output_settings={
        'type': 'localFile',
        'path': './winequality-white-predictions.csv'
    }
)

In [31]:
!type winequality-white-predictions.csv

quality_PREDICTION,DEPLOYMENT_APPROVAL_STATUS,wine_id
6.2146047619,APPROVED,100
6.5579769841,APPROVED,101
5.6517484127,APPROVED,102
6.4505222222,APPROVED,103
5.8184150794,APPROVED,104
5.6443966089,APPROVED,105
6.5156111111,APPROVED,106
4.8477015873,APPROVED,107
4.6883087302,APPROVED,108
5.345681746,APPROVED,109
